# LatentSim!

In [1]:
import numpy as np
from latent_sim import LatentSim

In [2]:
ls = LatentSim()

In [3]:
hub = "/Users/afq/Google Drive/networks/training_water_slgc_logp/"

In [4]:
ls.load_model(hub+"Classifying_2,1,12,24,sigmoid",
             scale_file="data_files/water_iapw_logp_ranges.csv",logp=True)

INFO:tensorflow:Restoring parameters from /Users/afq/Google Drive/networks/training_water_slgc_logp/Classifying_2,1,12,24,sigmoid/final_variables


In [5]:
ls.decode(np.array([[0.0,0.0]]))

array([[4.0347299e+02, 2.6940397e+05, 6.8761157e+02, 1.1177512e+06]],
      dtype=float32)

# Finding initial conditions

We know we only need 2 variables to specify a point on the surface. Most of the time we just want to provide a p and a T. 
However, the econder assumes that the input is on the manifold. 
The user could go through the effort of looking up rho and h, but that's a pain!
We want to let the user specify only 2 numbers and find the intersection on the manifold.

If we just try to plug in the result with "reasonable" fillers for the two unknowns we notice that we encoded onto the wrong point on the manifold:

In [6]:
q = ls.encode(np.array([[250.0, 1.0e3, ls.scale[0,2],ls.scale[0,3]]]))
print q
print ls.decode(q)

[[-0.22080615  0.4094363 ]]
[[2.8302521e+02 1.1946631e+03 5.1899634e+02 1.1841102e+06]]


This means:
\begin{equation}
T,p \neq D_{Tp}(E(T,p,?,?))
\end{equation}
Unfortunately this means we have to start solving the equations.
\begin{equation}
R(q) = D_{Tp}(q) - \{T^*,p^*\}
\end{equation}
with a good initial guess of
\begin{equation}
q^0 = E(T^*,p^*,?,?)
\end{equation}

In [7]:
ls.encode(
    np.array([[2.50000000e+02, 1.00000000e+03, 5.41351758e+02 ,1.35349597e+06]]) )

array([[-0.23225012,  0.40448084]], dtype=float32)

We can wrap this process up in a nice interface,

In [8]:
q0 = ls.find_point(T=250,p=1.0e3)

3.610995413128499 [[-0.22080615  0.4094363 ]] [1.50958045 3.28031321]
[0.41805106 0.90842353]
22.272882870021395 [[0.19724491 1.3178598 ]] [ -3.62571497 -21.97579355]
[-0.16278607 -0.98666139]
25.855246822248553 [[0.03445885 0.3311984 ]] [ 9.55475327 24.02499694]
[0.36954794 0.92921166]
14.27537806559748 [[0.40400678 1.2604101 ]] [ -2.83174568 -13.9916988 ]
[-0.19836572 -0.98012807]
38.91004022414984 [[0.20564106 0.280282  ]] [-14.38403845 -36.15370891]
[-0.36967421 -0.92916144]
0.2983209431675112 [[-0.16403314 -0.64887947]] [-0.21937555  0.20216269]
[-0.21937555  0.20216269]
0.04127537945130006 [[-0.3834087 -0.4467168]] [0.00326539 0.04114601]
[0.00326539 0.04114601]
0.04525518683242024 [[-0.3801433  -0.40557078]] [-0.02317911  0.03886851]
[-0.02317911  0.03886851]
0.04528573871928376 [[-0.40332243 -0.36670226]] [-0.02337551  0.03878639]
[-0.02337551  0.03878639]
0.045218263568219605 [[-0.42669794 -0.32791588]] [-0.02331296  0.03874528]
[-0.02331296  0.03874528]
0.0451453521056536 [[-

Note that we're working in 32-bit precision, so that's as close as we're going to get!

# Building the equations

In [9]:
ls.build_dae()

# Running a simulation

In [10]:
def schedule(t):
    if t<1000.0:
        ls.set_params(T_inf=800,p_inf=1.0e3)
    elif t<2000.0:
        ls.set_params(T_inf=800,p_inf=3.0e7)
    elif t<3000.0:
        ls.set_params(T_inf=400,p_inf=3.0e7)
    elif t<4000.0:
        ls.set_params(T_inf=400,p_inf=1.0e3)

In [ ]:
Dt = 0.1
ls.set_params(Dt=Dt)
ls.set_params(k_p=10.0,k_T=10000.0)
t = 0
t_max = 200.0
q = q0.copy()
qs,ss,ts = [],[],[]
while t<t_max:
    t+=Dt
    schedule(t)
    q = ls.solve_a_time_step(q)
    if np.isnan(q).any() or np.isinf(q).any(): break
    qs.append(q.copy())
    ss.append(ls.decode(q).copy())
    ts.append(t)

In [ ]:
ts = np.array(ts)
ss = np.array(ss)
qs = np.array(qs)

In [ ]:
from matplotlib import pylab as plt
%matplotlib inline
legends=['T','p','rho','h']
for i,name in enumerate(legends):
    plt.figure()
    plt.plot(ts,ss[:,0,i])
    plt.ylabel(name)